In [1]:
import scipy.io as scio
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import pylab
#from plotly.subplots import make_subplots
from tqdm import tqdm
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from utils import solve
import plotly.graph_objects as go
import torch
from baseline_NN import Net

/Users/bb/opt/anaconda3/envs/dr/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = scio.loadmat(f'dataset/test_K3.mat')
i = 2
power = data['data'][i]
price = data['price'][i]

params = scio.loadmat(f'result_data/OPT_K3_noise2_seed0_04102127.mat')
u = np.array(params['u'])
M = np.array(params['M'])[0]
L = np.array(params['L'])
params2 = scio.loadmat(f'result_data/IO_K3_noise2_04111255.mat')
u2 = np.array(params2['u'])
M2 = np.array(params2['M'])[0]
L2 = np.array(params2['L'])

x_pred = solve(price,u,M,L,3)
x_pred = x_pred.sum(axis=0)

x_pred2 = solve(price,u2,M2,L2,3)
x_pred2 = x_pred2.sum(axis=0)

In [3]:
model = Net()
model.load_state_dict(torch.load('result_data/NN_K3_noise2_seed0_04111327.pth'))
model.eval()
x_pred3 = model(torch.tensor(price).to(torch.float)).detach().numpy()


In [9]:
T = 12
N = 3

t1 = go.Scatter(x=np.arange(12), y=power,
                    mode='lines',
                    name='True',line = {'width':4.5})
t2 = go.Scatter(x=np.arange(12), y=x_pred,
                    mode='lines',
                    name='Ours',line={
        "width": 3})
t3 = go.Scatter(x=np.arange(12), y=x_pred2,
                    mode='lines',
                    name='IO',line={
        "width": 2})

t4 = go.Scatter(x=np.arange(12), y=x_pred3,
                    mode='lines',
                    name='NN',line={
        "width": 2})
t5 = go.Scatter(x=np.arange(T), y=price, yaxis='y2',
                    mode='lines',
                    name='price',line={
        "width": 3,'dash':'dash'})

fig = go.Figure(data=[t1,t2,t3,t4,t5],layout={
    'xaxis':{'title':'Hour', 'showline':True},
    'yaxis':{'title': 'load of the consumer (kWh)','range':[0,27]},
    'yaxis2': {'title':'price ($/kWh)','anchor':'x', 'overlaying':'y', 'side':'right', 'range':[0,60]},
    "font": {"size": 40,'color':'#3a4142'},
    'template':'plotly_white',
    'legend':{'yanchor':'bottom','y':1,'xanchor':'left','x':0.0,'orientation':'h'},
    'width':1600,
    'height':900
    
})
fig.show()

In [10]:
fig.write_image("result_image/comparison_K3.pdf")